In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier

In [2]:
# Set the random seed for reproducibility
random.seed(42)

In [3]:
# Load Dataset
file_path = "/Users/Sebastiano/ML_MRI copy.xlsx"
df = pd.read_excel(file_path)

df = shuffle(df)
df.reset_index(inplace=True, drop=True)

print("N° of patients: {}".format(len(df)))
print("N° of columns: {}".format(df.shape[1]))
df.head()

N° of patients: 47
N° of columns: 932


,Patient,Gender,Age,NP-SLE,Event,Scale factor,SNR,White Matter (WM) volume cm3,White Matter (WM) volume %,Normal Appearing White Matter volume cm3,...,FO left thickness mm,FO left thickness norm.,FO thickness asymmetry,PO total thickness mm,PO total thickness norm.,PO right thickness mm,PO right thickness norm.,PO left thickness mm,PO left thickness norm.,PO thickness asymmetry
0,Paziente 14,0,32,1,Mood abnormalities (depressive),0.62393,48.8158,410.6583,36.4750,410.5940,...,3.08680,0.029672,16.7448,2.23570,0.021491,2.30790,0.022185,2.16180,0.020781,6.5364
1,Paziente 7,0,44,0,Na,0.73629,46.8150,471.8149,35.2601,471.5572,...,2.57520,0.023369,11.4351,2.44380,0.022177,2.47710,0.022479,2.40860,0.021857,2.8044
2,job1628636,0,31,2,Na,0.82141,35.3838,432.9105,30.9426,417.0705,...,3.54490,0.031695,-4.8160,2.73770,0.024477,2.37400,0.021226,2.96800,0.026537,-22.2387
3,Paziente 20,0,30,1,"Seizure, ACS, myelitis, CVA",0.72170,100.8101,498.0416,38.5017,492.7147,...,0.99542,0.009136,1.3129,0.93666,0.008596,0.99354,0.009118,0.87294,0.008012,12.9221
4,Paziente 11,0,32,0,Na,0.68846,51.0280,481.9302,38.3537,481.5288,...,3.33130,0.030871,6.7254,2.07540,0.019233,2.03080,0.018820,2.10960,0.019550,-3.8050


In [4]:
# Drop unwanted columns
df = df.drop(['Patient', 'Gender', 'Age', 'Event', 'Scale factor', 'SNR'], axis='columns')

# Normalize the selected features
features_to_normalize = df.columns.difference(['NP-SLE'])
scaler = MinMaxScaler()
df[features_to_normalize] = scaler.fit_transform(df[features_to_normalize])


In [5]:
# Separate features and target variable
X = df.drop(['NP-SLE'], axis=1)
y = df['NP-SLE']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Initialize and train the XGBoost model
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', random_state=42)
xgb_clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softmax', ...)

In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# Define the hyperparameter grid to search
param_grid = {
    'C': [0.1, 1, 10, 100, 1000],  
    'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
    'kernel': ['linear', 'rbf', 'poly'],
    'degree': [2, 3, 4],  # Applicable for 'poly' kernel
}

# Create the SVM classifier
svm_classifier = SVC(probability=True, random_state=42)

# Initialize StratifiedKFold with 5 folds
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize the GridSearchCV with StratifiedKFold
grid_search = GridSearchCV(svm_classifier, param_grid, cv=stratified_kfold)

# Fit the GridSearchCV to your training data
grid_search.fit(X_train, y_train)

# Get the best parameters found by grid search
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Get the best SVM classifier with the best hyperparameters
best_svm_classifier = grid_search.best_estimator_

# Train the best SVM classifier on the training data
best_svm_classifier.fit(X_train, y_train)

Best Hyperparameters: {'C': 0.1, 'degree': 2, 'gamma': 1, 'kernel': 'poly'}


SVC(C=0.1, degree=2, gamma=1, kernel='poly', probability=True, random_state=42)

In [8]:
rf = RandomForestClassifier(n_estimators=500)
rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=500)

In [9]:
from sklearn.ensemble import StackingClassifier

# Define the base learners
base_learners = [
    ('svm', best_svm_classifier),
    ('rf', rf),
    ('xgb', xgb_clf)
]

# Initialize StackingClassifier
stacking_clf = StackingClassifier(
    estimators=base_learners, final_estimator=RandomForestClassifier()
)

# Train the StackingClassifier
stacking_clf.fit(X_train, y_train)

# Evaluate the model
y_pred_stacking = stacking_clf.predict(X_test)
accuracy_stacking = accuracy_score(y_test, y_pred_stacking)
report_stacking = classification_report(y_test, y_pred_stacking, zero_division=0)

print("Accuracy on Test Set:", accuracy_stacking)
print("Classification Report:\n", report_stacking)


Accuracy on Test Set: 0.6
Classification Report:
               precision    recall  f1-score   support

           0       0.25      0.50      0.33         2
           1       0.50      0.25      0.33         4
           2       1.00      1.00      1.00         4

    accuracy                           0.60        10
   macro avg       0.58      0.58      0.56        10
weighted avg       0.65      0.60      0.60        10

